In [105]:
import pandas as pd
import re
import itertools

In [129]:
info = pd.read_csv('../data/sources/WeimarDB/weimar_db_melody_info.csv')

In [134]:
info.drop_duplicates('chord_changes')['title'].value_counts().head(80)

Body and Soul              14
Anthropology                4
All the Things You Are      4
I'll Remember April         4
Dickie's Dream              3
                           ..
Star Eyes                   2
Limehouse Blues             2
Did You Call Her Today      2
Segment                     2
Scrapple from the Apple     2
Name: title, Length: 80, dtype: int64

In [284]:
all_cp = {}

for i, row in info.drop_duplicates('chord_changes').iterrows():
    title = row['title'] + ' --- ' + str(row['melid'])
    if title.split('---')[0].strip() in chord_prog.keys():
        c = row['chord_changes']

        parts = c.split('\n')
        song_structure = {
            'sections': [],
            'progression': {},
            'key': row['key']
        }

        for part in parts:
            pn = part.split(':')[0]

            cp = part.split(':')[1].strip().replace('||', '').split('|')

    #         print(cp)
            chord_progression = []

            for m in cp:
                m = m.replace('NC', ' ')
                cs = re.split(r'([ ,A-Z][a-z,#,0-9,-]*)', m)
    #             print(cs)

                for c in cs:
                    if c != '' and c != " ":
                        chord = c

                    if c != '':
                        chord_progression.append(chord)

            song_structure['progression'][pn] = chord_progression 

        all_cp[title] = song_structure

In [285]:
list(all_cp.keys())

['Anthropology --- 1',
 'Desafinado --- 3',
 'I Got It Bad --- 7',
 'Just Friends --- 10',
 'Sweet Lorraine --- 13',
 'Avalon --- 14',
 'Bye Bye Blackbird --- 21',
 'My Ideal --- 23',
 'Margie --- 27',
 "I Didn't Know What Time It Was --- 33",
 'Three Little Words --- 42',
 'High Fly --- 47',
 'So What --- 48',
 'Star Eyes --- 49',
 'Work Song --- 51',
 "Billie's Bounce --- 52",
 "Don't Blame Me --- 54",
 'Donna Lee --- 55',
 'Embraceable You --- 56',
 'Ko-Ko --- 59',
 'My Little Suede Shoes --- 60',
 'Ornithology --- 61',
 'Segment --- 64',
 'Star Eyes --- 65',
 'Yardbird Suite --- 68',
 'Limehouse Blues --- 69',
 'Just Friends --- 71',
 "Let's Get Lost --- 72",
 'There Will Never Be Another You --- 74',
 'There Will Never Be Another You --- 75',
 'Anthropology --- 78',
 'Daahoud --- 88',
 "I'll Remember April --- 90",
 "I'll Remember April --- 91",
 'Jordu --- 92',
 'Joy Spring --- 93',
 'Sandu --- 94',
 'Perdido --- 99',
 'Sophisticated Lady --- 100',
 'Blue Train --- 102',
 "I've G

In [319]:
for k, v in all_cp.items():
    title = k.split(' --- ')[0]
    melid = int(k.split(' --- ')[1])
    
    filter_info = info[info['melid'] == melid]
        
#     print(chord_prog[k]['sections'])
#     print(chord_prog[k]['progression'])
    
    for i, group in filter_info.groupby('melid'):
        sections = list(group['form'].drop(['I1'], errors='ignore').dropna())
        try:
            sections.remove('I1')
        except:
            pass
        try:
            sections.remove('I1')
        except:
            pass
        try:
            sections.remove('I1')
        except:
            pass
        try:
            sections.remove('I')
        except:
            pass
        try:
            sections.remove('I')
        except:
            pass
                
    all_cp[k]['sections'] = sections
        
#         print(sections)
    
#     print('-----------')
    
#     ccs = filter_info.drop_duplicates('chord_changes')['chord_changes'].values
# 
#     if len(ccs) > 1:
#         for cc1, cc2 in list(itertools.combinations(ccs, 2)):
# #             sm = difflib.SequenceMatcher(None, cc1, cc2)
# #             s = sm.ratio()
#             n_measures1 = len(cc1.replace(r'[A-Z][0-9]\: ||', '').replace('||', '').split('|'))
#             n_measures2 = len(cc2.replace(r'[A-Z][0-9]\: ||', '').replace('||', '').split('|'))

#             if n_measures1 != n_measures2:
#                 print(k)
#                 print(n_measures1, n_measures2)

#                 for cc in filter_info.drop_duplicates('chord_changes')['chord_changes'].values:
#                     print(cc)
#                     print('\n')
        

#                 print('-----------------')
#                 print('\n')

In [321]:
from utils import get_chord_progressions
import difflib

In [322]:
chord_prog = get_chord_progressions('..')

In [338]:
all_cp['Ko-Ko --- 59']['progression'].keys()

dict_keys(['B1', 'A3'])

In [336]:
info[info['title'] == 'Ko-Ko']['form'].dropna()

15249     I1
15253    *B1
15317     A3
15381     A1
15445     A2
15509     B1
15573     A3
15637     A1
Name: form, dtype: object

In [327]:
print(list(all_cp['Ko-Ko --- 59']['progression'].keys()))
print(all_cp['Ko-Ko --- 59']['sections'])

['B1', 'A3']
['*B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1']


In [374]:
for k, cp in all_cp.items():
    k2 = k.split('---')[0].strip()
    if k2 in chord_prog.keys():
        linear_chord_progression = []
        linear_chord_progression_n = []
        
        ncp = chord_prog[k2]
        
        try: 
            for section in ncp['sections']:
                linear_chord_progression_n += ncp['progression'][section]

            for section in cp['sections']:
                linear_chord_progression += cp['progression'][section.replace('*', '')]

#             if ncp['key'][0] == cp['key'][0]:

            sm = difflib.SequenceMatcher(None, linear_chord_progression, linear_chord_progression_n)
            s = sm.ratio()

            if len(linear_chord_progression) != len(linear_chord_progression_n):       
                if len(linear_chord_progression) % len(linear_chord_progression_n) > 64:
                    print(k)
                    print(s)

    #                     print(len(linear_chord_progression_n))
    #                     print(len(linear_chord_progression))
                    print(len(linear_chord_progression) % len(linear_chord_progression_n))
                    print(ncp['sections'])
                    print(cp['sections'])


    #                     for i in range(min(len(linear_chord_progression), len(linear_chord_progression_n))):
    #                         if linear_chord_progression[i][0] != linear_chord_progression_n[i][0]:
    #                             print(i, linear_chord_progression[i], linear_chord_progression_n[i])

                    print('------------')

        except Exception as e:
            print('###################')
            print(e)
            print('###################')
            


Desafinado --- 3
0.0
240
['A', 'A2', 'B', 'A3']
['A1', 'A2', 'B1', 'C1', 'A1']
------------
I Got It Bad --- 7
0.01812688821752266
69
['A', 'N1', 'A', 'N2', 'B', 'A2']
['A1', 'A2', 'B1', 'A3', 'B2', 'A4']
------------
Three Little Words --- 42
0.07258064516129033
96
['A', 'A', 'B', 'A2']
['A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1', 'A3', 'A1', 'A2', 'B1']
------------
###################
'A1'
###################
There Will Never Be Another You --- 75
0.05357142857142857
96
['A', 'B', 'A2', 'C']
['A1', 'B1', 'A2']
------------
Joy Spring --- 93
0.056818181818181816
96
['A', 'B', 'C', 'A2']
['A1', 'B1', 'A2', 'A1', 'B1']
------------
Blue Train --- 102
0.04081632653061224
80
['A1', 'A2', 'B1', 'A3']
['A1', 'A1', 'A1', 'A1', 'A1', 'A1']
------------
Nica's Dream --- 108
0.0
256
['A', 'ToCod', 'A', 'ToCod', 'B', 'A', 'ToCod', 'A', 'Coda']
['A1', 'A2', 'B1', 'A3']
------------
Chelsea Bridge --- 12

In [347]:
48 % 111

48